### Download data

In [9]:
from datasets import load_dataset

ds_train = load_dataset("re_dial", revision="refs/convert/parquet", split="train")
ds_test = load_dataset("re_dial", revision="refs/convert/parquet", split="test")

ds_train.to_pandas().to_parquet("data/raw/train.parquet")
ds_test.to_pandas().to_parquet("data/raw/test.parquet")

### Verify Interim

In [39]:
import pandas as pd

interim_path = 'data/processed/translated_ds_011.parquet'

df = pd.read_parquet(interim_path)
print(f'total de linhas: {len(df)}')

# non_translated_count = df['text_translated'].isna().sum()
# translated_count = len(df) - non_translated_count

# print(f'Total de mensagens traduzidas: {translated_count}')
# print(f'Total de mensagens não traduzidas: {non_translated_count}')


total de linhas: 1342


In [26]:
def reaggregate_messages_and_translation(group):

    messages = group.apply(
        lambda x: {
            "timeOffset": x['messages']["timeOffset"],
            "text": x['messages']["text"],
            "senderWorkerId": x['messages']["senderWorkerId"],
            "messageId": x['messages']["messageID"],
        },
        axis=1,
    ).tolist()

    messages_translated = group.apply(
        lambda x: {
            "timeOffset": x["timeOffset"],
            "text": x["text_translated"],
            "senderWorkerId": x["senderWorkerId"],
            "messageId": x["messageID"],
        },
        axis=1,
    ).tolist()

    # Colunas constantes dentro de cada grupo
    constant_values = {
        "movieMentions": group["movieMentions"].iloc[0],
        "respondentQuestions": group["respondentQuestions"].iloc[0],
        "respondentWorkerId": group["respondentWorkerId"].iloc[0],
        "initiatorWorkerId": group["initiatorWorkerId"].iloc[0],
        "initiatorQuestions": group["initiatorQuestions"].iloc[0],
    }

    return pd.Series(
        {
            "messages": messages,
            "messages_translated": messages_translated,
            **constant_values,
        }
    )


def reconstruct_dataset(df_messages: pd.DataFrame, number: int) -> pd.DataFrame:
    reconstructed_df = (
        df_messages.groupby("conversationId")
        .apply(reaggregate_messages_and_translation)
        .reset_index()
    )
    reconstructed_df.to_parquet(
        f"data/processed/translated_ds_{number:03}.parquet", index=False
    )

### Split Dataset

In [1]:
import pandas as pd
import numpy as np

# Carregando o dataset

PATH = 'data/raw/ds.parquet'

df = pd.read_parquet(PATH)

# Dividindo o dataframe em 10 partes
splits = np.array_split(df, 10)

# Salvando cada parte como um arquivo parquet separado
for idx, split in enumerate(splits, 1):
    filename = f'data/raw/ds_{idx:03}.parquet'
    split.to_parquet(filename, index=False)
    print(f"Saved {filename}")


Saved data/raw/ds_001.parquet
Saved data/raw/ds_002.parquet
Saved data/raw/ds_003.parquet
Saved data/raw/ds_004.parquet
Saved data/raw/ds_005.parquet
Saved data/raw/ds_006.parquet
Saved data/raw/ds_007.parquet
Saved data/raw/ds_008.parquet
Saved data/raw/ds_009.parquet
Saved data/raw/ds_010.parquet


### Example generating batchs samples

In [98]:
movies = pd.DataFrame(df['movieMentions'].explode().to_list()).drop_duplicates()

In [183]:
import re
def id2text(text):
    for movie_id in re.findall(r'@\d+', text):
        movie_name = movies[movies['movieId'] == movie_id[1:]]['movieName'].iloc[0]
        text = text.replace(movie_id, movie_name)
    return text

In [184]:
df_dev = df.head()
df_dev = df_dev[['messages_translated']]
df_dev['conversation'] = df_dev['messages_translated'].apply(lambda x: [id2text(message['text']) for message in x])
df_dev.drop(columns=['messages_translated'], inplace=True)

In [185]:
# split question x response explode
df_train = []

for _, row in df_dev.iterrows():
    for i in range(len(row['conversation']) - 1):
        df_train.append({'question': row['conversation'][i], 'response': row['conversation'][i+1]})

df_train = pd.DataFrame(df_train)
df_train

,question,response
0,"Olá, estou procurando por um filme como o Super Troopers (2001).",Você deveria assistir Police Academy (1984).
1,Você deveria assistir Police Academy (1984).,É um grande? Eu nunca vi isso. Eu já vi American Pie .
2,É um grande? Eu nunca vi isso. Eu já vi American Pie .,Eu quero dizer American Pie (1999)
3,Eu quero dizer American Pie (1999),"Sim, Police Academy (1984) é muito engraçado e Police Academy 2: Their First Assignment (1985) também."
4,"Sim, Police Academy (1984) é muito engraçado e Police Academy 2: Their First Assignment (1985) também.",Parece que eu preciso dar uma olhada neles.
...,...,...
74,Acho que vou tentar esse.,X-Men: The Last Stand (2006) Foi outro bom filme dos X-Men.
75,X-Men: The Last Stand (2006) Foi outro bom filme dos X-Men.,"Okay, obrigado!"
76,"Okay, obrigado!","Oh, tem tantas coisas"
77,"Oh, tem tantas coisas",Obrigado.


In [164]:
df_dev.iloc[0]

conversation    [Olá, estou procurando por um filme como o Super Troopers (2001)., Você deveria assistir Police Academy  (1984)., É um grande? Eu nunca vi isso. Eu já vi American Pie ., Eu quero dizer American Pie  (1999), Sim, Police Academy  (1984) é muito engraçado e Police Academy 2: Their First Assignment (1985) também., Parece que eu preciso dar uma olhada neles., Sim, você vai gostar deles., Eu agradeço seu tempo. Eu precisarei dar uma olhada nisso. Existem outros que você recomendaria?, Sim Lethal Weapon (1987), Obrigado, eu também vou assistir isso., E também Beverly Hills Cop (1984), Obrigado pelas sugestões., Você é bem-vindo, E também 48 Hrs. (1982), Obrigado, adeus.]
Name: 0, dtype: object

In [176]:
df_dev

,conversation,question,response
0,"Olá, estou procurando por um filme como o Super Troopers (2001).","Olá, estou procurando por um filme como o Super Troopers (2001).",Você deveria assistir Police Academy (1984).
0,Você deveria assistir Police Academy (1984).,Você deveria assistir Police Academy (1984).,É um grande? Eu nunca vi isso. Eu já vi American Pie .
0,É um grande? Eu nunca vi isso. Eu já vi American Pie .,É um grande? Eu nunca vi isso. Eu já vi American Pie .,Eu quero dizer American Pie (1999)
0,Eu quero dizer American Pie (1999),Eu quero dizer American Pie (1999),"Sim, Police Academy (1984) é muito engraçado e Police Academy 2: Their First Assignment (1985) também."
0,"Sim, Police Academy (1984) é muito engraçado e Police Academy 2: Their First Assignment (1985) também.","Sim, Police Academy (1984) é muito engraçado e Police Academy 2: Their First Assignment (1985) também.",Parece que eu preciso dar uma olhada neles.
...,...,...,...
4,X-Men: The Last Stand (2006) Foi outro bom filme dos X-Men.,X-Men: The Last Stand (2006) Foi outro bom filme dos X-Men.,"Okay, obrigado!"
4,"Okay, obrigado!","Okay, obrigado!","Oh, tem tantas coisas"
4,"Oh, tem tantas coisas","Oh, tem tantas coisas",Obrigado.
4,Obrigado.,Obrigado.,Adeus


In [158]:
pd.set_option('display.max_colwidth', None)
df_dev

,conversation
0,"[Olá, estou procurando por um filme como o Super Troopers (2001)., Você deveria assistir Police Academy (1984)., É um grande? Eu nunca vi isso. Eu já vi American Pie ., Eu quero dizer American Pie (1999), Sim, Police Academy (1984) é muito engraçado e Police Academy 2: Their First Assignment (1985) também., Parece que eu preciso dar uma olhada neles., Sim, você vai gostar deles., Eu agradeço seu tempo. Eu precisarei dar uma olhada nisso. Existem outros que você recomendaria?, Sim Lethal Weapon (1987), Obrigado, eu também vou assistir isso., E também Beverly Hills Cop (1984), Obrigado pelas sugestões., Você é bem-vindo, E também 48 Hrs. (1982), Obrigado, adeus.]"
1,"[Olá!, Olá!, Qual tipo de filmes você gosta?, Procuro uma recomendação de filme. Quando eu era mais jovem, eu realmente gostei muito do A Nightmare on Elm Street (1984)., Oh, você gosta de filmes de terror?, Recentemente, eu assisti Happy Death Day (2017)., Eu também gostei de assistir The Last House on the Left (1972), Foi realmente bom para um novo ""filme de terror""., Eu gosto de alguns dos filmes de terror mais recentes que eu vi também., Ouvi dizer que o A Quiet Place (2018) é bom. Ele ainda está em cartaz nos cinemas, no entanto., Eu gostei muito do filme., Eu também!, Era realmente assustador, mas eu achei bom!, Eu vi esse filme no cinema, ele foi muito bom. Ele me deixou na ponta da cadeira durante todo o filme., Não tenho certeza se vi aquilo, preciso verificar. Parece familiar, mas não tenho certeza. Obrigado pelas suas sugestões!, Existem algumas comédias que você sugere?, Claro! Eu gosto muito de comédias. Eu gosto de filmes como Hot Tub Time Machine e Super Troopers (2001), mas eu também gosto de Identity Thief (2013) e The Heat (2013)., Maravilhoso! Muito obrigado, acho que estou pronto para a noite de cinema agora., Sem problemas! Obrigado também! :)]"
2,"[Me diga o que você gostaria de assistir?, Easy A (2010), Você assistiu ao Avengers: Infinity War (2018)?, Or The Black Panthers: Vanguard of the Revolution (2015) ?, Eu realmente gostei disso., Eu acho que eu gostaria de assistir Easy A (2010)., É engraçado, Mas você já sabe o que quer assistir, Você deveria me dizer que tipo de filmes você gosta., Você assistiu ao Click (2006)?, Você gosta de filmes de terror?, Você assistiu ao It (2017)?, Sim, eu gosto do Adam Sandler. O The Waterboy (1998) é outro bom filme dele., Eu não vi o filme., Sim, acho que sim., Espero ter ajudado., Adeus]"
3,"[Eu gostaria de assistir a qualquer filme., Me diga qualquer filme., Gostei do seu post Avengers: Infinity War (2018)., Você viu The Avengers (2012)?, Eu realmente gostei daquele., Sim e eu gostei disso., Qual outro você me recomendaria?, Você gosta de coisas assustadoras?, Você viu It (2017)?, Eu assisti ao vídeo Iron Man 2 (2010) e gostei dele., Ou Cabin by the Lake (2000), Não, eu não vi It (2017)., Mas eu vou tentar., Obrigado por me ajudar., Adeus, Se você gosta de super-heróis, o Ant-Man (2015) também é uma ótima opção., Adeus]"
4,"[Qual tipo de filmes você gosta?, Você viu o Avengers: Infinity War (2018)?, Ou Spider-Man (2002), Eu também gostei deles., Essas são incríveis., Eu gosto desse tipo de filmes, X-Men: First Class (2011), Eu também assisti Iron Man (2008)., Esses também são bons filmes., Não, eu não assisti aquele., Acho que vou tentar esse., X-Men: The Last Stand (2006) Foi outro bom filme dos X-Men., Okay, obrigado!, Oh, tem tantas coisas, Obrigado., Adeus]"


In [ ]:
def generate_sample(conversation):
    results = []
    for message in conversation:
        processed_example = "<|system|>\n Você é um chatbot de recomendação de filmes, converse com o usuário para indicar filmes apropriados.</s>\n<|user|>\n" + example[self.config.INSTRUCTION_FIELD] + "</s>\n<|assistant|>\n" + example[self.config.TARGET_FIELD]
